# Práctico 1

En este práctico, correspondiente a la primera mitad de la materia, trabajaremos con un dataset extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [30]:
import pandas as pd
from pathlib import Path
import numpy as np

# ## Descargar el csv con los datos en este directorio
# DATA_DIR = Path('../data/twitter')
# INPUT_FILE = DATA_DIR / 'vacunas.csv'

# ## Creamos el directorio en caso de que no exista
# DATA_DIR.mkdir(parents=True, exist_ok=True)

### Cargamos el dataset

In [3]:
dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}

path='../data/vacunas.csv'
df = pd.read_csv(path, dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)
df.head()

(155123, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN


### Observamos algunos ejemplos

In [4]:
idx = 0
print('Texto:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Texto: Seguimos esperando el comunicado de @norabar repudiando la situación respecto del gobierno y el tema vacunas. Seamos pacientes que con esto de la pandemia anda con mucho "laburo".
Retweets: 9


In [5]:
idx = 376
print('Text:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Text: Todo lo que hay que entender sobre la decisión –o no– de poner más vacunas en más brazos (por ejemplo, usar las 1º dosis en muchos y si es necesario retrasar la 2º) está en esta excelente nota de Nora Bär. https://t.co/A0I03DyxgO
Retweets: 48


### Calculamos la cantidad de hilos

In [6]:
roots = df[df['in_reply_to_user_id'].isna()]
roots.shape

(3174, 8)

## Actividades

### **1. Construcción del grafo** 

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.


### **2. Centralidad**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

### **3. Comunidades**

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada. 

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

# Actividad 1: Construcción del grafo

In [4]:
df_enlaces=df[['user_retweeters','user.id']].explode('user_retweeters').reset_index(drop=True)
df_enlaces.drop_duplicates(inplace=True)   # Se eliminan enlaces duplicados
df_enlaces.dropna(inplace=True)

In [5]:
import networkx as nx

g_di = nx.from_pandas_edgelist(df_enlaces, source='user_retweeters', target='user.id',create_using=nx.DiGraph) 
print(nx.info(g_di))

Name: 
Type: DiGraph
Number of nodes: 39800
Number of edges: 93692
Average in degree:   2.3541
Average out degree:   2.3541


In [6]:
# Verificamos si la cantidad de usuarios coincide con el número de nodos. En la columna user_id no aparecen todos los nodos, por ejemplo un usuario que retweeteo 
# pero que no participó de los hilos.

len(set(df_enlaces['user.id'].tolist() + df_enlaces['user_retweeters'].tolist()))

39800

## Versión no dirigida

In [7]:
g = nx.from_pandas_edgelist(df_enlaces, source='user_retweeters', target='user.id',create_using=nx.Graph) 
print(nx.info(g))

Name: 
Type: Graph
Number of nodes: 39800
Number of edges: 93404
Average degree:   4.6937


### Matriz de adyacencia

In [19]:
A = nx.adjacency_matrix(g)
A=A.toarray()
print(A)

[[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Matriz Laplaciana

La conectividad se puede estudiar en términos de los autovalores de la matriz laplaciana (diferencia entre la matriz de grado y la de adyacencia L=D-A).

El segundo autovalor no nulo se denomina conectividad algebraica y es una medida de la conectividad del grafo (mas pequeño, mas modular)

In [22]:
L = nx.laplacian_matrix(g).toarray()
print(L)

[[19 -1  0 ...  0  0  0]
 [-1  9 -1 ...  0  0  0]
 [ 0 -1  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  1]]


Podemos tambien usar la funcion algebraic_connectivity de Network x (es el segundo autovalor no nulo de la matriz Laplaciana)

In [32]:
nx.algebraic_connectivity(g)

0.0

Vemos que no es un grafo conexo, por lo cual nos quedamos con la componente gigante.

In [36]:
Gcc = sorted(nx.connected_components(g), key=len, reverse=True)
G = g.subgraph(Gcc[0])

In [37]:
nx.algebraic_connectivity(G)

0.014577204930859393

In [41]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 38998
Number of edges: 92830
Average degree:   4.7608


In [49]:
df_metricas=pd.DataFrame(G.degree()).rename(columns={0:'nodo',1:'degree'})

clustering_coefficient = nx.clustering(G)
df_metricas['Cws'] = list(clustering_coefficient.values())